In [1]:
import os, random, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

In [11]:
train_dir = 'data'
validation_dir = 'validation'


In [14]:
total_train = 0
for c in ['elliptical', 'uncertain', 'spiral']:
  total_train += len(os.listdir(os.path.join(train_dir, c)))
print('Total train:', total_train)

total_validation = 0
for c in ['elliptical', 'uncertain', 'spiral']:
  total_validation += len(os.listdir(os.path.join(validation_dir, c)))
print('Total validation:', total_validation)

Total train: 37604
Total validation: 9416


In [15]:
# Create the generators for the training and validation data
# Data Augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1.0/255,
                    rotation_range=25,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.2)
validation_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.)

# Use the flow_from_directory() method with the appropriate directory and
# the lists of images for the training and validation sets
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size=34,
                                                    shuffle=True,
                                                    class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                         target_size=(150,150),
                                                         batch_size=34,
                                                         shuffle=False,
                                                         class_mode='categorical')

Found 37604 images belonging to 3 classes.
Found 9416 images belonging to 3 classes.


In [16]:
#Convolutional neural network model

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(150, 150, 3)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 67500)             0         
                                                                 
 dense (Dense)               (None, 256)               17280256  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 17,321,603
Trainable params: 17,321,603
Non-trainable params: 0
_________________________________________________________________


In [17]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

hist = model.fit(train_generator,
                    epochs=200,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/200
1106/1106 [==============================] - 442s 399ms/step - loss: 0.8180 - accuracy: 0.6391 - val_loss: 0.7135 - val_accuracy: 0.6713
Epoch 2/200
1106/1106 [==============================] - 311s 281ms/step - loss: 0.7074 - accuracy: 0.6755 - val_loss: 0.6885 - val_accuracy: 0.6862
Epoch 3/200
1106/1106 [==============================] - 302s 273ms/step - loss: 0.6768 - accuracy: 0.6922 - val_loss: 0.6881 - val_accuracy: 0.6813
Epoch 4/200
1106/1106 [==============================] - 304s 275ms/step - loss: 0.6665 - accuracy: 0.6982 - val_loss: 0.6863 - val_accuracy: 0.6788
Epoch 5/200
1106/1106 [==============================] - 300s 271ms/step - loss: 0.6576 - accuracy: 0.7017 - val_loss: 0.6566 - val_accuracy: 0.7050
Epoch 6/200
1106/1106 [==============================] - 297s 268ms/step - loss: 0.6449 - accuracy: 0.7074 - val_loss: 0.6586 - val_accuracy: 0.6916
Epoch 7/200
1106/1106 [==============================] - 297s 269ms/step - loss: 0.6403 - accuracy: 0.7136

In [33]:
# Deep neural network

modelb = tf.keras.Sequential()

modelb.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
modelb.add(tf.keras.layers.MaxPooling2D((2, 2)))
modelb.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
modelb.add(tf.keras.layers.MaxPooling2D((2, 2)))
modelb.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
modelb.add(tf.keras.layers.MaxPooling2D((2, 2)))
modelb.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
modelb.add(tf.keras.layers.MaxPooling2D((2, 2)))
modelb.add(tf.keras.layers.Flatten())
modelb.add(tf.keras.layers.Dense(512, activation='relu'))
modelb.add(tf.keras.layers.Dense(3, activation='softmax'))
modelb.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelb.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)     

In [34]:
modelb.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

hist_b = modelb.fit(train_generator,
                    epochs=200,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/200
1106/1106 [==============================] - 541s 489ms/step - loss: 0.6936 - accuracy: 0.6782 - val_loss: 0.6308 - val_accuracy: 0.7209
Epoch 2/200
1106/1106 [==============================] - 550s 497ms/step - loss: 0.6034 - accuracy: 0.7267 - val_loss: 0.6407 - val_accuracy: 0.7141
Epoch 3/200
1106/1106 [==============================] - 534s 483ms/step - loss: 0.5860 - accuracy: 0.7378 - val_loss: 0.5731 - val_accuracy: 0.7508
Epoch 4/200
1106/1106 [==============================] - 533s 481ms/step - loss: 0.5716 - accuracy: 0.7448 - val_loss: 0.5700 - val_accuracy: 0.7484
Epoch 5/200
1106/1106 [==============================] - 534s 483ms/step - loss: 0.5605 - accuracy: 0.7515 - val_loss: 0.5723 - val_accuracy: 0.7463
Epoch 6/200
1106/1106 [==============================] - 578s 522ms/step - loss: 0.5564 - accuracy: 0.7533 - val_loss: 0.5624 - val_accuracy: 0.7528
Epoch 7/200
1106/1106 [==============================] - 746s 674ms/step - loss: 0.5499 - accuracy: 0.7542

In [37]:
#ResNet18 is 18 deep layer model

from tensorflow.keras.applications import InceptionV3

modelc = InceptionV3(include_top=True, weights=None, input_tensor=None, input_shape=(150,150,3), classes=3)
modelc.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelc.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d_4[0][0]']               
 alization)                                                                                       
                                                                                       

 batch_normalization_10 (BatchN  (None, 16, 16, 96)  288         ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 16, 16, 32)  96          ['conv2d_15[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_5 (Activation)      (None, 16, 16, 64)   0           ['batch_normalization_5[0][0]']  
                                                                                                  
 activation_7 (Activation)      (None, 16, 16, 64)   0           ['batch_normalization_7[0][0]']  
                                                                                                  
 activatio

 conv2d_24 (Conv2D)             (None, 16, 16, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_27 (Conv2D)             (None, 16, 16, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 16, 16, 48)  144         ['conv2d_24[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_23 (BatchN  (None, 16, 16, 96)  288         ['conv2d_27[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                                                  
 conv2d_38 (Conv2D)             (None, 7, 7, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 7, 7, 128)   384         ['conv2d_38[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 7, 7, 128)    0           ['batch_normalization_34[0][0]'] 
                                                                                                  
 conv2d_39 (Conv2D)             (None, 7, 7, 128)    114688      ['activation_34[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 activation_44 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_49 (Conv2D)             (None, 7, 7, 160)    179200      ['activation_44[0][0]']          
                                                                                                  
 batch_normalization_45 (BatchN  (None, 7, 7, 160)   480         ['conv2d_49[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_45 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_45[0][0]'] 
                                                                                                  
 conv2d_45

 batch_normalization_55 (BatchN  (None, 7, 7, 160)   480         ['conv2d_59[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 7, 7, 160)    0           ['batch_normalization_55[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 7, 7, 160)    122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_60 (Conv2D)             (None, 7, 7, 160)    179200      ['activation_55[0][0]']          
                                                                                                  
 batch_normalization_51 (BatchN  (None, 7, 7, 160)   480         ['conv2d_55[0][0]']              
 ormalizat

 conv2d_65 (Conv2D)             (None, 7, 7, 192)    147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_70 (Conv2D)             (None, 7, 7, 192)    258048      ['activation_65[0][0]']          
                                                                                                  
 batch_normalization_61 (BatchN  (None, 7, 7, 192)   576         ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_66 (BatchN  (None, 7, 7, 192)   576         ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_74 (BatchN  (None, 7, 7, 192)   576         ['conv2d_78[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_70 (Activation)     (None, 7, 7, 192)    0           ['batch_normalization_70[0][0]'] 
                                                                                                  
 activation_74 (Activation)     (None, 7, 7, 192)    0           ['batch_normalization_74[0][0]'] 
                                                                                                  
 conv2d_75 (Conv2D)             (None, 3, 3, 320)    552960      ['activation_70[0][0]']          
          

                                                                                                  
 activation_82 (Activation)     (None, 3, 3, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 3, 3, 384)    0           ['batch_normalization_83[0][0]'] 
                                                                                                  
 batch_normalization_84 (BatchN  (None, 3, 3, 192)   576         ['conv2d_88[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_76 (Activation)     (None, 3, 3, 320)    0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 mixed9_0 

 ormalization)                                                                                    
                                                                                                  
 activation_85 (Activation)     (None, 3, 3, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 (Concatenate)         (None, 3, 3, 768)    0           ['activation_87[0][0]',          
                                                                  'activation_88[0][0]']          
                                                                                                  
 concatenate_1 (Concatenate)    (None, 3, 3, 768)    0           ['activation_91[0][0]',          
                                                                  'activation_92[0][0]']          
                                                                                                  
 activatio

In [38]:
hist_c = modelc.fit(train_generator,
                    epochs=200,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/200
1106/1106 [==============================] - 1927s 2s/step - loss: 0.7105 - accuracy: 0.6789 - val_loss: 0.6809 - val_accuracy: 0.6983
Epoch 2/200
1106/1106 [==============================] - 1923s 2s/step - loss: 0.6294 - accuracy: 0.7152 - val_loss: 0.8668 - val_accuracy: 0.6702
Epoch 3/200
1106/1106 [==============================] - 1904s 2s/step - loss: 0.5990 - accuracy: 0.7303 - val_loss: 0.6473 - val_accuracy: 0.7019
Epoch 4/200
1106/1106 [==============================] - 1938s 2s/step - loss: 0.5842 - accuracy: 0.7378 - val_loss: 1.1910 - val_accuracy: 0.6429
Epoch 5/200
1106/1106 [==============================] - 1930s 2s/step - loss: 0.5799 - accuracy: 0.7433 - val_loss: 0.5936 - val_accuracy: 0.7259
Epoch 6/200
1106/1106 [==============================] - 1948s 2s/step - loss: 0.5690 - accuracy: 0.7455 - val_loss: 1.9827 - val_accuracy: 0.4739
Epoch 7/200
1106/1106 [==============================] - 1934s 2s/step - loss: 0.5659 - accuracy: 0.7476 - val_loss: 0

In [66]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Instantiate the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet',
                         include_top=False,
                         input_shape=(150,150,3))

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Add new layers to the model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

# Define the new model
modeld = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
modeld.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modeld.summary()

87910968/87910968 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_192 (Conv2D)            (None, 74, 74, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 74, 74, 32)  96          ['conv2d_192[0][0]']             
 Normalization)                                                                                   
                             

                                                                                                  
 batch_normalization_195 (Batch  (None, 16, 16, 64)  192         ['conv2d_199[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_198 (Batch  (None, 16, 16, 96)  288         ['conv2d_202[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_199 (Batch  (None, 16, 16, 32)  96          ['conv2d_203[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 Normalization)                                                                                   
                                                                                                  
 activation_210 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_210[0][0]']
                                                                                                  
 conv2d_212 (Conv2D)            (None, 16, 16, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_215 (Conv2D)            (None, 16, 16, 96)   55296       ['activation_210[0][0]']         
                                                                                                  
 batch_normalization_208 (Batch  (None, 16, 16, 48)  144         ['conv2d_212[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 mixed3 (Concatenate)           (None, 7, 7, 768)    0           ['activation_214[0][0]',         
                                                                  'activation_217[0][0]',         
                                                                  'max_pooling2d_14[0][0]']       
                                                                                                  
 conv2d_226 (Conv2D)            (None, 7, 7, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_222 (Batch  (None, 7, 7, 128)   384         ['conv2d_226[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 conv2d_236 (Conv2D)            (None, 7, 7, 160)    122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_232 (Batch  (None, 7, 7, 160)   480         ['conv2d_236[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_232 (Activation)    (None, 7, 7, 160)    0           ['batch_normalization_232[0][0]']
                                                                                                  
 conv2d_237 (Conv2D)            (None, 7, 7, 160)    179200      ['activation_232[0][0]']         
                                                                                                  
 batch_nor

                                                                                                  
 activation_242 (Activation)    (None, 7, 7, 160)    0           ['batch_normalization_242[0][0]']
                                                                                                  
 conv2d_247 (Conv2D)            (None, 7, 7, 160)    179200      ['activation_242[0][0]']         
                                                                                                  
 batch_normalization_243 (Batch  (None, 7, 7, 160)   480         ['conv2d_247[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_243 (Activation)    (None, 7, 7, 160)    0           ['batch_normalization_243[0][0]']
                                                                                                  
 conv2d_24

 batch_normalization_253 (Batch  (None, 7, 7, 192)   576         ['conv2d_257[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_253 (Activation)    (None, 7, 7, 192)    0           ['batch_normalization_253[0][0]']
                                                                                                  
 conv2d_253 (Conv2D)            (None, 7, 7, 192)    147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_258 (Conv2D)            (None, 7, 7, 192)    258048      ['activation_253[0][0]']         
                                                                                                  
 batch_normalization_249 (Batch  (None, 7, 7, 192)   576         ['conv2d_253[0][0]']             
 Normaliza

 conv2d_262 (Conv2D)            (None, 7, 7, 192)    147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_266 (Conv2D)            (None, 7, 7, 192)    258048      ['activation_261[0][0]']         
                                                                                                  
 batch_normalization_258 (Batch  (None, 7, 7, 192)   576         ['conv2d_262[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_262 (Batch  (None, 7, 7, 192)   576         ['conv2d_266[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 batch_normalization_264 (Batch  (None, 3, 3, 320)   960         ['conv2d_268[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_266 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_266[0][0]']
                                                                                                  
 activation_267 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_267[0][0]']
                                                                                                  
 activation_270 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_270[0][0]']
                                                                                                  
 activation_271 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_271[0][0]']
          

                                                                                                  
 activation_279 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_279[0][0]']
                                                                                                  
 activation_280 (Activation)    (None, 3, 3, 384)    0           ['batch_normalization_280[0][0]']
                                                                                                  
 batch_normalization_281 (Batch  (None, 3, 3, 192)   576         ['conv2d_285[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_273 (Activation)    (None, 3, 3, 320)    0           ['batch_normalization_273[0][0]']
                                                                                                  
 mixed9_1 

In [67]:
class_weights = {0: 1.,
                1: 1.5,
                2: 1.}

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

hist_d = model.fit(train_generator, 
          epochs=200, 
          validation_data=validation_generator,
          class_weight = class_weights,
          callbacks=[early_stopping])

Epoch 1/200
1106/1106 [==============================] - 327s 295ms/step - loss: 0.6905 - accuracy: 0.7313 - val_loss: 0.6046 - val_accuracy: 0.7288
Epoch 2/200
1106/1106 [==============================] - 299s 270ms/step - loss: 0.6882 - accuracy: 0.7330 - val_loss: 0.6177 - val_accuracy: 0.7172
Epoch 3/200
1106/1106 [==============================] - 299s 270ms/step - loss: 0.6855 - accuracy: 0.7302 - val_loss: 0.6149 - val_accuracy: 0.7150
Epoch 4/200
1106/1106 [==============================] - 299s 271ms/step - loss: 0.6863 - accuracy: 0.7314 - val_loss: 0.6010 - val_accuracy: 0.7245
Epoch 5/200
1106/1106 [==============================] - 299s 270ms/step - loss: 0.6840 - accuracy: 0.7328 - val_loss: 0.6165 - val_accuracy: 0.7191
Epoch 6/200
1106/1106 [==============================] - 302s 273ms/step - loss: 0.6861 - accuracy: 0.7305 - val_loss: 0.6340 - val_accuracy: 0.7051
Epoch 7/200
1106/1106 [==============================] - 313s 283ms/step - loss: 0.6831 - accuracy: 0.7307

In [71]:
from tensorflow.keras.preprocessing import image
import numpy as np
# Load the image
img = image.load_img('C:/Users/3196s/OneDrive/Desktop/Python/gmclassification/154554.jpg', target_size=(150, 150))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Add an extra dimension to the array for the batch dimension
x = np.expand_dims(x, axis=0)

# Normalize the image data
x = x / 255.

# Get the model's predictions
predictions = model.predict(x)

# Print the predictions
print('Model prediction:', predictions[0])

if predictions[0][0] > predictions[0][1] and predictions[0][0] > predictions[0][2]:
    print('Model prediction: [1, 0, 0]. Its an Elliptical galaxy')
elif predictions[0][1] > predictions[0][0] and predictions[0][1] > predictions[0][2]:
    print('Model prediction: [0, 1, 0]. Its an Spiral galaxy')
else:
    print('Model prediction: [0, 0, 1]. Its an Uncertain galaxy')
predictions = modelb.predict(x)
print('Model_B prediction:', predictions[0])
if predictions[0][0] > predictions[0][1] and predictions[0][0] > predictions[0][2]:
    print('Model_B prediction: [1, 0, 0]. Its an Elliptical galaxy')
elif predictions[0][1] > predictions[0][0] and predictions[0][1] > predictions[0][2]:
    print('Model_B prediction: [0, 1, 0]. Its an Spiral galaxy')
elif predictions[0][2] > predictions[0][0] and predictions[0][2] > predictions[0][1]:
    print('Model_B prediction: [0, 0, 1]. Its an Uncertain galaxy')
    
predictions = modelc.predict(x)
print('Model_C prediction:', predictions[0])
if predictions[0][0] > predictions[0][1] and predictions[0][0] > predictions[0][2]:
    print('Model_C prediction: [1, 0, 0]. Its an Elliptical galaxy')
elif predictions[0][1] > predictions[0][0] and predictions[0][1] > predictions[0][2]:
    print('Model_C prediction: [0, 1, 0]. Its an Spiral galaxy')
elif predictions[0][2] > predictions[0][0] and predictions[0][2] > predictions[0][1]:
    print('Model_C prediction: [0, 0, 1]. Its an Uncertain galaxy')

1/1 [==============================] - 1s 605ms/step
Model prediction: [6.3649177e-06 1.3095427e-02 9.8689818e-01]
Model prediction: [0, 0, 1]. Its an Uncertain galaxy
1/1 [==============================] - 0s 90ms/step
Model_B prediction: [1.4234330e-03 9.7460434e-04 9.9760193e-01]
Model_B prediction: [0, 0, 1]. Its an Uncertain galaxy
1/1 [==============================] - 1s 659ms/step
Model_C prediction: [0.00223147 0.00861117 0.9891573 ]
Model_C prediction: [0, 0, 1]. Its an Uncertain galaxy


In [70]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the directory containing the images
img_dir = 'C:/Users/3196s/OneDrive/Desktop/Python/gmclassification/new try/data2/'

# Define the directories for saving the images based on predictions
elliptical_dir = 'C:/Users/3196s/OneDrive/Desktop/Python/gmclassification/new try/elliptical/'
spiral_dir = 'C:/Users/3196s/OneDrive/Desktop/Python/gmclassification/new try/spiral/'
uncertain_dir = 'C:/Users/3196s/OneDrive/Desktop/Python/gmclassification/new try/uncertain/'

# Loop through all the images in the directory
for filename in os.listdir(img_dir):
    # Load the image
    img = image.load_img(img_dir + filename, target_size=(150, 150))

    # Convert the image to a numpy array
    x = image.img_to_array(img)

    # Add an extra dimension to the array for the batch dimension
    x = np.expand_dims(x, axis=0)

    # Normalize the image data
    x = x / 255.

    # Get the predictions from the first model
    predictions = modelb.predict(x)

    # Check the predictions and save the image in the appropriate directory
    if predictions[0][0] > predictions[0][1] and predictions[0][0] > predictions[0][2]:
        image.save_img(elliptical_dir + filename, img)
        #print(f'{filename} is an elliptical galaxy')
    elif predictions[0][1] > predictions[0][0] and predictions[0][1] > predictions[0][2]:
        image.save_img(spiral_dir + filename, img)
        #print(f'{filename} is a spiral galaxy')
    else:
        image.save_img(uncertain_dir + filename, img)
        #print(f'{filename} is an uncertain galaxy')

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 35ms/step


...

1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 37ms/step
